In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import re
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('reviews.csv')
df.shape

(48136, 15)

In [4]:
df.columns

Index(['age', 'body type', 'bust size', 'category', 'fit', 'height', 'item_id',
       'rating', 'rented for', 'review_date', 'review_summary', 'review_text',
       'size', 'user_id', 'weight'],
      dtype='object')

In [5]:
df.dtypes

age               float64
body type          object
bust size          object
category           object
fit                object
height             object
item_id             int64
rating            float64
rented for         object
review_date        object
review_summary     object
review_text        object
size                int64
user_id             int64
weight             object
dtype: object

In [6]:
df.describe(include="all")

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight
count,47894.000000,44544,43567,48136,48136,47952,4.813600e+04,48116.000000,48133,48136,48136,48136,48136.000000,48136.000000,40594
unique,NaN,7,95,62,3,24,NaN,NaN,8,2120,41417,47935,NaN,NaN,173
top,NaN,hourglass,34b,dress,fit,"5' 4""",NaN,NaN,wedding,"June 15, 2016",b'Stylist Review',b'.',NaN,NaN,130lbs
freq,NaN,13887,6732,23179,35430,7100,NaN,NaN,14411,208,230,18,NaN,NaN,3576
mean,33.866580,NaN,NaN,NaN,NaN,NaN,1.043875e+06,9.084255,NaN,NaN,NaN,NaN,12.248255,499091.944470,NaN
std,8.018446,NaN,NaN,NaN,NaN,NaN,8.067556e+05,1.436092,NaN,NaN,NaN,NaN,8.523070,288853.095706,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,1.233730e+05,2.000000,NaN,NaN,NaN,NaN,0.000000,47.000000,NaN
25%,29.000000,NaN,NaN,NaN,NaN,NaN,1.941820e+05,8.000000,NaN,NaN,NaN,NaN,8.000000,249516.000000,NaN
50%,32.000000,NaN,NaN,NaN,NaN,NaN,9.458800e+05,10.000000,NaN,NaN,NaN,NaN,12.000000,498811.000000,NaN
75%,37.000000,NaN,NaN,NaN,NaN,NaN,1.678888e+06,10.000000,NaN,NaN,NaN,NaN,16.000000,749178.750000,NaN


In [7]:
df.drop("review_summary", axis=1, inplace=True)
df.drop("review_text", axis=1, inplace=True)

df.dropna(subset=['age', 'body type', 'bust size', 'category', 'fit', 'height', 'item_id','rating', 'rented for', 'review_date','size', 'user_id', 'weight'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df.describe(include="all")

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,size,user_id,weight
count,36677.000000,36677,36677,36677,36677,36677,3.667700e+04,36677.000000,36677,36677,36677.000000,36677.000000,36677
unique,NaN,7,88,61,3,24,NaN,NaN,8,2066,NaN,NaN,169
top,NaN,hourglass,34b,dress,fit,"5' 4""",NaN,NaN,wedding,"June 15, 2016",NaN,NaN,130lbs
freq,NaN,11051,5873,17625,26928,5513,NaN,NaN,10575,153,NaN,NaN,3237
mean,34.084822,NaN,NaN,NaN,NaN,NaN,1.050619e+06,9.078060,NaN,NaN,11.468632,498566.634485,NaN
std,8.032131,NaN,NaN,NaN,NaN,NaN,8.109600e+05,1.442064,NaN,NaN,7.857180,289344.763863,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,1.233730e+05,2.000000,NaN,NaN,0.000000,47.000000,NaN
25%,29.000000,NaN,NaN,NaN,NaN,NaN,1.942320e+05,8.000000,NaN,NaN,4.000000,249049.000000,NaN
50%,32.000000,NaN,NaN,NaN,NaN,NaN,9.543030e+05,10.000000,NaN,NaN,9.000000,498238.000000,NaN
75%,38.000000,NaN,NaN,NaN,NaN,NaN,1.687082e+06,10.000000,NaN,NaN,16.000000,748988.000000,NaN


In [9]:
df["body type"].unique()

array(['athletic', 'full bust', 'petite', 'hourglass',
       'straight & narrow', 'pear', 'apple'], dtype=object)

In [10]:
df["bust size"].unique()

array(['32ddd/e', '36d', '34b', '34c', '36c', '32b', '36aa', '32c', '36b',
       '34dd', '32d', '36a', '34d', '38ddd/e', '38d', '34a', '38c',
       '36ddd/e', '34d+', '32a', '34ddd/e', '32d+', '30dd', '34g', '38b',
       '40dd', '32dd', '34h', '36d+', '36h', '36dd', '30c', '34aa', '44b',
       '38dd', '30d', '32g', '38d+', '34f', '36g', '28dd', '32aa', '32f',
       '28a', '30ddd/e', '40d', '30b', '38a', '30a', '44g', '32h', '38f',
       '42g', '28c', '42ddd/e', '42d', '42dd', '36f', '40f', '28aa',
       '38g', '30f', '28d', '42b', '30g', '40c', '42c', '40ddd/e', '38aa',
       '30h', '28ddd/e', '36i', '28b', '44d', '40h', '40g', '44f', '32j',
       '38i', '28f', '36j', '30aa', '44c', '44ddd/e', '38j', '38h',
       '44dd', '40b'], dtype=object)

In [11]:
df["category"].unique()

array(['sweater', 'gown', 'dress', 'sheath', 'maxi', 'romper', 'top',
       'mini', 'shirtdress', 'blouse', 'jumpsuit', 'shift', 'pants',
       'coat', 'culotte', 'jacket', 'tank', 'culottes', 'skirt', 'tunic',
       'blazer', 'sweatshirt', 'down', 'frock', 'vest', 'overalls',
       'skirts', 'cape', 'cardigan', 'bomber', 'shirt', 'suit', 'henley',
       'hoodie', 'poncho', 'for', 'kimono', 'blouson', 'pullover',
       'trousers', 'turtleneck', 'kaftan', 'pant', 't-shirt', 'ballgown',
       'knit', 'legging', 'print', 'trench', 'cami', 'leggings', 'duster',
       'trouser', 'tee', 'midi', 'peacoat', 'combo', 'skort', 'parka',
       'buttondown', 'crewneck'], dtype=object)

In [12]:
df["fit"].unique()

array(['fit', 'large', 'small'], dtype=object)

In [13]:
df["height"].unique()

array(['5\' 6"', '5\' 5"', '5\' 3"', '5\' 7"', '5\' 4"', '5\' 1"',
       '5\' 2"', '5\' 8"', '5\' 10"', '5\' 9"', '5\' 11"', '5\' 0"',
       '4\' 11"', '6\' 2"', '6\' 0"', '4\' 10"', '6\' 1"', '4\' 9"',
       '4\' 8"', '4\' 7"', '4\' 6"', '6\' 3"', '6\' 6"', '6\' 4"'],
      dtype=object)

In [14]:
df["rented for"].unique()

array(['everyday', 'wedding', 'work', 'other', 'formal affair', 'party',
       'date', 'vacation'], dtype=object)

In [15]:
df["weight"].unique()

array(['140lbs', '128lbs', '132lbs', '125lbs', '150lbs', '110lbs',
       '134lbs', '120lbs', '130lbs', '117lbs', '145lbs', '170lbs',
       '135lbs', '137lbs', '100lbs', '138lbs', '127lbs', '165lbs',
       '142lbs', '147lbs', '141lbs', '236lbs', '160lbs', '156lbs',
       '108lbs', '210lbs', '121lbs', '155lbs', '118lbs', '115lbs',
       '168lbs', '104lbs', '136lbs', '190lbs', '200lbs', '185lbs',
       '103lbs', '133lbs', '175lbs', '50lbs', '195lbs', '123lbs',
       '151lbs', '105lbs', '129lbs', '99lbs', '230lbs', '144lbs',
       '112lbs', '205lbs', '119lbs', '116lbs', '101lbs', '172lbs',
       '111lbs', '180lbs', '124lbs', '126lbs', '178lbs', '188lbs',
       '174lbs', '122lbs', '158lbs', '109lbs', '98lbs', '90lbs', '143lbs',
       '106lbs', '131lbs', '148lbs', '146lbs', '113lbs', '161lbs',
       '95lbs', '114lbs', '149lbs', '169lbs', '220lbs', '159lbs',
       '196lbs', '152lbs', '163lbs', '107lbs', '189lbs', '167lbs',
       '153lbs', '206lbs', '166lbs', '157lbs', '201lbs', 

In [20]:
def stripLbs(string):
    string = string[:-3]
    return int(string.strip())
    
df["weight"] = df["weight"].apply(stripLbs)

TypeError: 'int' object is not subscriptable

In [17]:
df.describe(include="all") #someone is only 50 pounds??

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,size,user_id,weight
count,36677.000000,36677,36677,36677,36677,36677,3.667700e+04,36677.000000,36677,36677,36677.000000,36677.000000,36677.000000
unique,NaN,7,88,61,3,24,NaN,NaN,8,2066,NaN,NaN,NaN
top,NaN,hourglass,34b,dress,fit,"5' 4""",NaN,NaN,wedding,"June 15, 2016",NaN,NaN,NaN
freq,NaN,11051,5873,17625,26928,5513,NaN,NaN,10575,153,NaN,NaN,NaN
mean,34.084822,NaN,NaN,NaN,NaN,NaN,1.050619e+06,9.078060,NaN,NaN,11.468632,498566.634485,137.299643
std,8.032131,NaN,NaN,NaN,NaN,NaN,8.109600e+05,1.442064,NaN,NaN,7.857180,289344.763863,21.610244
min,0.000000,NaN,NaN,NaN,NaN,NaN,1.233730e+05,2.000000,NaN,NaN,0.000000,47.000000,50.000000
25%,29.000000,NaN,NaN,NaN,NaN,NaN,1.942320e+05,8.000000,NaN,NaN,4.000000,249049.000000,123.000000
50%,32.000000,NaN,NaN,NaN,NaN,NaN,9.543030e+05,10.000000,NaN,NaN,9.000000,498238.000000,135.000000
75%,38.000000,NaN,NaN,NaN,NaN,NaN,1.687082e+06,10.000000,NaN,NaN,16.000000,748988.000000,148.000000


In [28]:
def heightToMeters(string):
    string = string.replace(" ", "")
    pattern = r'([0-9])\'(.*)\"'
    reg = re.search(pattern, string)
    feet = int(reg.group(1))
    inches = int(reg.group(2))
    return 0.0254 * ((feet * 12) + inches)
    
df["height"] = df["height"].apply(heightToMeters)

In [29]:
df.describe(include="all")

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,size,user_id,weight
count,36677.000000,36677,36677,36677,36677,36677.000000,3.667700e+04,36677.000000,36677,36677,36677.000000,36677.000000,36677.000000
unique,NaN,7,88,61,3,NaN,NaN,NaN,8,2066,NaN,NaN,NaN
top,NaN,hourglass,34b,dress,fit,NaN,NaN,NaN,wedding,"June 15, 2016",NaN,NaN,NaN
freq,NaN,11051,5873,17625,26928,NaN,NaN,NaN,10575,153,NaN,NaN,NaN
mean,34.084822,NaN,NaN,NaN,NaN,1.657547,1.050619e+06,9.078060,NaN,NaN,11.468632,498566.634485,137.299643
std,8.032131,NaN,NaN,NaN,NaN,0.067261,8.109600e+05,1.442064,NaN,NaN,7.857180,289344.763863,21.610244
min,0.000000,NaN,NaN,NaN,NaN,1.371600,1.233730e+05,2.000000,NaN,NaN,0.000000,47.000000,50.000000
25%,29.000000,NaN,NaN,NaN,NaN,1.600200,1.942320e+05,8.000000,NaN,NaN,4.000000,249049.000000,123.000000
50%,32.000000,NaN,NaN,NaN,NaN,1.651000,9.543030e+05,10.000000,NaN,NaN,9.000000,498238.000000,135.000000
75%,38.000000,NaN,NaN,NaN,NaN,1.701800,1.687082e+06,10.000000,NaN,NaN,16.000000,748988.000000,148.000000
